In [ ]:
# Imports
import pandas as pd
import numpy as np

df_players = pd.read_csv('dataset/processed/players_teams_processed.csv')
df_teams = pd.read_csv('dataset/processed/teams_processed.csv')
df_awards = pd.read_csv('dataset/processed/awards_processed.csv')

league_df_stats = pd.DataFrame()
league_df_stats['lg_AST'] = df_teams.groupby('year')['o_asts'].sum().reset_index(drop=True)
league_df_stats['lg_FG'] = df_teams.groupby('year')['o_fgm'].sum().reset_index(drop=True)
league_df_stats['lg_FT'] = df_teams.groupby('year')['o_ftm'].sum().reset_index(drop=True)
league_df_stats['lg_PTS'] = df_teams.groupby('year')['o_pts'].sum().reset_index(drop=True)
league_df_stats['lg_FGA'] = df_teams.groupby('year')['o_fga'].sum().reset_index(drop=True)
league_df_stats['lg_ORB'] = df_teams.groupby('year')['o_oreb'].sum().reset_index(drop=True)
league_df_stats['lg_TOV'] = df_teams.groupby('year')['o_to'].sum().reset_index(drop=True) 
league_df_stats['lg_FTA'] = df_teams.groupby('year')['o_fta'].sum().reset_index(drop=True) 
league_df_stats['lg_TRB'] = df_teams.groupby('year')['o_reb'].sum().reset_index(drop=True) 
league_df_stats['lg_PF'] = df_teams.groupby('year')['o_pf'].sum().reset_index(drop=True) 

stats_df_players = pd.merge(df_players, df_teams, on=['tmID', 'year'], how='left')

for i in range(10):
  stats_df_players.loc[stats_df_players['year'] == i+1,'factor'] = (2/3) - (0.5* (league_df_stats['lg_AST'].iloc[i] / league_df_stats['lg_FG'].iloc[i])) / (2 * (league_df_stats['lg_FG'].iloc[i] / league_df_stats['lg_FT'].iloc[i]))
  stats_df_players.loc[stats_df_players['year'] == i+1,'vop'] = league_df_stats['lg_PTS'].iloc[i] / (league_df_stats['lg_FGA'].iloc[i] - league_df_stats['lg_ORB'].iloc[i] + league_df_stats['lg_TOV'].iloc[i] + 0.44 * league_df_stats['lg_FTA'].iloc[i])
  stats_df_players.loc[stats_df_players['year'] == i+1,'drb'] = (league_df_stats['lg_TRB'].iloc[i] - league_df_stats['lg_ORB'].iloc[i]) / league_df_stats['lg_TRB'].iloc[i]
#stats_df_players['PER'] = (1/ (df_players['minutes'] + df_players['PostMinutes'])) * ((df_players['threeMade'] + df_players['PostthreeMade']) + 2/3 * (df_players['assists']+ df_players['PostAssists']) + (2 - stats_df_players['factor'] *  (stats_df_players['o_asts'] / stats_df_players['o_fgm'])) * (stats_df_players['fgMade'] + stats_df_players['PostfgMade']) + ((stats_df_players['ftMade'] + stats_df_players['PostftMade']) * 0.5 * (1 + (1 - (stats_df_players['o_asts'] / stats_df_players['o_fgm'])) + (2/3) * (stats_df_players['o_asts'] / stats_df_players['o_fgm']))) - stats_df_players['vop'] * (stats_df_players['turnovers'] + stats_df_players['PostTurnovers']) - stats_df_players['vop'] * stats_df_players['drb'] * ((stats_df_players['fgAttempted'] + stats_df_players['PostfgAttempted']) - (stats_df_players['fgMade'] + stats_df_players['PostfgMade'])) - stats_df_players['vop'])

  stats_df_players.loc[stats_df_players['year'] == i+1,'PER'] = (
    (1 / (df_players['minutes'] + df_players['PostMinutes'])).replace([np.inf, -np.inf, np.nan], 0) * 
    ((df_players['threeMade'] + df_players['PostthreeMade'])
      +  (2 / 3) * (df_players['assists'] + df_players['PostAssists'])
      +  (2 - stats_df_players['factor'] * (stats_df_players['o_asts'] / stats_df_players['o_fgm']).replace([np.inf, -np.inf, np.nan], 0)) * (stats_df_players['fgMade'] + stats_df_players['PostfgMade']) 
      +  ((stats_df_players['ftMade'] + stats_df_players['PostftMade']) * 0.5 * (1 + (1 - (stats_df_players['o_asts'] / stats_df_players['o_fgm']).replace([np.inf, -np.inf, np.nan], 0)) + (2 / 3) * (stats_df_players['o_asts'] / stats_df_players['o_fgm']).replace([np.inf, -np.inf, np.nan], 0))) 
      - stats_df_players['vop'] * (stats_df_players['turnovers'] + stats_df_players['PostTurnovers']) 
      - stats_df_players['vop'] * stats_df_players['drb'] * ((stats_df_players['fgAttempted'] + stats_df_players['PostfgAttempted']) - (stats_df_players['fgMade'] + stats_df_players['PostfgMade'])) 
      - stats_df_players['vop'] * 0.44 * (0.44 + (0.56 * stats_df_players['drb'])) * ((stats_df_players['ftAttempted'] + stats_df_players['PostftAttempted']) - (stats_df_players['ftMade'] + stats_df_players['PostftMade']))
      + stats_df_players['vop'] * (1 - stats_df_players['drb']) * (stats_df_players['rebounds'] - stats_df_players['oRebounds'])
      + stats_df_players['vop'] * stats_df_players['drb'] * stats_df_players['oRebounds']
      + stats_df_players['vop'] * (stats_df_players['steals'] * stats_df_players['PostSteals'])
      + stats_df_players['vop'] * stats_df_players['drb'] * (stats_df_players['blocks'] * stats_df_players['PostBlocks'])
      - (stats_df_players['PF'] + stats_df_players['PostPF']) * ((league_df_stats['lg_FT'].iloc[i] / league_df_stats['lg_PF'].iloc[i]) - 0.44 * (league_df_stats['lg_FTA'].iloc[i] / league_df_stats['lg_PF'].iloc[i]) * stats_df_players['vop'])
    )
)
minimumPer = -stats_df_players['PER'].min()
stats_df_players['PER'] = stats_df_players['PER'] + minimumPer + 1
stats_df_players['PER'] = stats_df_players.groupby('playerID', group_keys=False)['PER'].shift(1)
#CUMULATIVE PER
stats_df_players[stats_df_players['minutes'] + stats_df_players['PostMinutes'] < 100]['PER'] = 0

#stats_df_players = stats_df_players[['playerID', 'year', 'tmID', 'PER']].copy()
team_stats = (
    stats_df_players
    .sort_values(by=['tmID', 'year', 'PER'], ascending=[True, True, False])  # Sort within groups
    .groupby(['tmID', 'year'])  # Group by team ID and year
    .head(5)  # Take the top 5 players per team and year
    .groupby(['tmID', 'year'])['PER']  # Group again for PER calculation
    .mean()  # Calculate the mean of PER for the top 10 players
    .reset_index()  # Reset index for the result
)
team_stats['PER'] = team_stats.groupby('tmID', group_keys=False)['PER'].shift(-1)
team_stats['PER'] = team_stats['PER'].fillna(team_stats['PER'].mean())
print(team_stats[team_stats['tmID'] == "CHA"])
# print(stats_df_players.loc[(stats_df_players['minutes'] + stats_df_players['PostMinutes']) > 500].sort_values(by=['PER'], ascending=[False]).groupby(['tmID', 'year'])['PER', 'tmID', 'playerID'].head(10))

df_teams = pd.merge(df_teams, team_stats, on=['tmID', 'year'], how='left')
df_teams.to_csv('dataset/processed/teams_test.csv', index=False)


# Display the sorted DataFrame



        playerID  year  stint tmID  GP_x  GS  minutes  points  oRebounds  \
0     abrossv01w     2      0  MIN    26  23      846     343         43   
1     abrossv01w     3      0  MIN    27  27      805     314         45   
2     abrossv01w     4      0  MIN    30  25      792     318         44   
3     abrossv01w     5      0  MIN    22  11      462     146         17   
4     abrossv01w     6      0  MIN    31  31      777     304         29   
...          ...   ...    ...  ...   ...  ..      ...     ...        ...   
1871  zakalok01w     3      2  PHO     5   0       37       6          0   
1872   zarafr01w     6      0  SEA    34   4      413      90         11   
1873  zellosh01w    10      0  DET    34   4      802     406         25   
1874  zirkozu01w     4      0  WAS     6   0       30      11          0   
1875   zollsh01w     9      0  MIN     6   0       30      10          1   

      dRebounds  ...  awayW  awayL  confW  confL   min    factor       vop  \
0        

C:\Users\JoaoCoutinho\AppData\Local\Temp\ipykernel_16816\344376885.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_df_players = stats_df_players.groupby('playerID').apply(calculate_cumulative_PER).reset_index(drop=True)
C:\Users\JoaoCoutinho\AppData\Local\Temp\ipykernel_16816\344376885.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_df_players[stats_df_players['minutes'] + stats_df_players['PostMinutes'] < 100]['Cumulative PER'] = 0


In [194]:
df_teams = pd.read_csv('dataset/processed/teams_test.csv')
df_teams = df_teams.sort_values(by=['franchID', 'year'])
df_teams['playoffNextYear'] = df_teams['playoff'].shift(-1)
df_teams.loc[df_teams['franchID'] != df_teams['franchID'].shift(-1), 'playoffNextYear'] = None
df_teams.dropna(subset=['playoffNextYear'], inplace=True)
#df_teams["confW"] = df_teams["confW"].fillna(0)
#df_teams = df_teams.fillna(-1)
label_encoder = LabelEncoder()
df_teams['playoff'] = label_encoder.fit_transform(df_teams['playoff'])
df_teams['playoffNextYear'] = label_encoder.fit_transform(df_teams['playoffNextYear'])
features = ['PER', 'Cumulative Win Rate'] # attend?

df_teams_train = df_teams[df_teams['year'] <= 7]
df_teams_test = df_teams[df_teams['year'].isin([8,9])]
# df_teams_train = df_teams[df_teams['year'] <= 5]
# df_teams_test = df_teams[df_teams['year'].isin([6])]

training_inputs = df_teams_train[features]
training_classes = df_teams_train['playoffNextYear']
testing_inputs = df_teams_test[features]
testing_classes = df_teams_test['playoffNextYear']
print(df_teams_train[df_teams_train['tmID'] == "CHA"][['year', 'PER', 'coachID', 'Cumulative Win Rate', 'playoff', 'playoffNextYear']])

dt_class = DecisionTreeClassifier(random_state=42)
dt_class.fit(training_inputs, training_classes)
# print(training_inputs.isna().sum())
dt_class.predict(testing_inputs)

dt_class.score(testing_inputs, testing_classes)

accuracy_score(testing_classes, dt_class.predict(testing_inputs))

importance = dt_class.feature_importances_
for feature, imp in zip(features, importance):
    print(f"Feature: {feature}, Importance: {imp}")

'''f = open("results.txt", "a")
f.write("Year 9 Accuracy = " + str(accuracy_score(testing_classes, dt_class.predict(testing_inputs))) + "\n")
f.close()'''

data_results(testing_classes, testing_inputs, dt_class)

# plot_tree(dt_class, feature_names=features, filled=True)

NameError: name 'LabelEncoder' is not defined